In [ ]:
import os
import pandas as pd
import json
import gzip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
print(os.listdir())

['.config', 'drive', 'sample_data']


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import numpy as np


In [ ]:
from datasets import Dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("nikesh66/Sarcasm-dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/420 [00:00<?, ?B/s]

sarcasm_tweets.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
headline_df = pd.read_json(
    "/content/drive/MyDrive/99. Past/INST798_Project/Sarcasm_Headlines_Dataset.json",
    lines=True
)

In [ ]:
headline_df = headline_df.drop(columns=["article_link"])

In [ ]:
headline_df

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


In [ ]:
headline_df = headline_df[["headline", "is_sarcastic"]].copy()
headline_df = headline_df.rename(columns={"is_sarcastic": "label"})
headline_df["label"] = headline_df["label"].astype(int)

headline_df.head()

,headline,label
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
dataset = Dataset.from_pandas(headline_df)

# Train/validation split (10% for validation)
dataset = dataset.train_test_split(test_size=0.2)
train_ds = dataset["train"]
val_ds = dataset["test"]



In [ ]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    return tokenizer(
        batch["headline"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

train_ds = train_ds.map(preprocess, batched=True)
val_ds   = val_ds.map(preprocess, batched=True)

# Tell Trainer which columns are inputs
train_ds = train_ds.remove_columns(
    [c for c in train_ds.column_names if c not in ["input_ids", "attention_mask", "label"]]
)
val_ds = val_ds.remove_columns(
    [c for c in val_ds.column_names if c not in ["input_ids", "attention_mask", "label"]]
)

train_ds.set_format("torch")
val_ds.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

Map:   0%|          | 0/5342 [00:00<?, ? examples/s]

In [ ]:
id2label = {0: "not_sarcastic", 1: "sarcastic"}
label2id = {"not_sarcastic": 0, "sarcastic": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=preds, references=labels)["f1"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="sarcasm-roberta",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3457493100.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.205900,0.180021,0.932984,0.920831
2,0.151900,0.205963,0.941221,0.931261
3,0.071700,0.322291,0.932422,0.919004


TrainOutput(global_step=4008, training_loss=0.1811176407004069, metrics={'train_runtime': 1560.9633, 'train_samples_per_second': 41.065, 'train_steps_per_second': 2.568, 'total_flos': 4216420439907840.0, 'train_loss': 0.1811176407004069, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import classification_report

pred_output = trainer.predict(val_ds)

# Extract logits and true labels
logits = pred_output.predictions
y_true = pred_output.label_ids
y_pred = np.argmax(logits, axis=-1)

# Print classification report
print(classification_report(y_true, y_pred, target_names=["not_sarcastic", "sarcastic"]))

               precision    recall  f1-score   support

not_sarcastic       0.91      0.97      0.94      3016
    sarcastic       0.96      0.88      0.92      2326

     accuracy                           0.93      5342
    macro avg       0.94      0.93      0.93      5342
 weighted avg       0.93      0.93      0.93      5342



In [ ]:
non_sarcastic_examples = [
    "I’m excited to start my new job next week.",
    "The weather is beautiful today.",
    "I finally finished my project on time.",
    "She did a great job organizing the event.",
    "I love spending weekends with my family.",
    "The teacher explained the concept very clearly.",
    "I really appreciate your help.",
    "The restaurant served delicious food.",
    "My flight arrived earlier than expected.",
    "I enjoy reading books in the evening."
]
sarcastic_examples = [
    "Oh great, another meeting that could’ve been an email.",
    "Yeah, because waking up at 5 AM is exactly what I wanted.",
    "Fantastic, my phone died right before the interview. Perfect timing.",
    "I absolutely love waiting in line for an hour.",
    "Amazing, another update that breaks everything.",
    "Sure, I’ll totally drop everything just to fix your mistake.",
    "Wow, what a surprise, the Wi-Fi is terrible again.",
    "Yeah, because that’s exactly how physics works.",
    "Perfect, rain on the one day I planned a picnic.",
    "Oh look, the printer stopped working again. Shocking."
]
hard_examples = [
    "Well, that presentation was… something.",
    "Nice job. Really. Amazing.",
    "He’s a real genius, isn’t he?",
    "Guess I should’ve expected that from him.",
    "Love it when people talk over me.",
    "What a productive day. I did absolutely nothing.",
    "Oh awesome, they changed the deadline again.",
    "Yeah, because that’s never happened before.",
    "I’m sure this will end well.",
    "Great, now I get to redo all my work."
]

In [ ]:
model.save_pretrained("/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline")
tokenizer.save_pretrained("/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline")

('/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline/tokenizer_config.json',
 '/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline/special_tokens_map.json',
 '/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline/vocab.json',
 '/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline/merges.txt',
 '/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline/added_tokens.json',
 '/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline/tokenizer.json')

In [ ]:
from transformers import pipeline

model_path = "/content/drive/MyDrive/INST798_Project/finetuned-sarcasm-headline"

sarcasm_clf = pipeline(
    "text-classification",
    model=model_path,
    tokenizer=model_path
)

Device set to use cuda:0


In [ ]:
for text in non_sarcastic_examples:
    print(text)
    print(sarcasm_clf(text))
    print("-----")

I’m excited to start my new job next week.
[{'label': 'not_sarcastic', 'score': 0.9996135830879211}]
-----
The weather is beautiful today.
[{'label': 'not_sarcastic', 'score': 0.9995898604393005}]
-----
I finally finished my project on time.
[{'label': 'not_sarcastic', 'score': 0.9995038509368896}]
-----
She did a great job organizing the event.
[{'label': 'not_sarcastic', 'score': 0.9995642304420471}]
-----
I love spending weekends with my family.
[{'label': 'not_sarcastic', 'score': 0.9996291399002075}]
-----
The teacher explained the concept very clearly.
[{'label': 'not_sarcastic', 'score': 0.9991114735603333}]
-----
I really appreciate your help.
[{'label': 'not_sarcastic', 'score': 0.9995235204696655}]
-----
The restaurant served delicious food.
[{'label': 'not_sarcastic', 'score': 0.9992414712905884}]
-----
My flight arrived earlier than expected.
[{'label': 'not_sarcastic', 'score': 0.9995788931846619}]
-----
I enjoy reading books in the evening.
[{'label': 'not_sarcastic', 'sc

In [ ]:
for text in sarcastic_examples:
    print(text)
    print(sarcasm_clf(text))
    print("-----")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Oh great, another meeting that could’ve been an email.
[{'label': 'not_sarcastic', 'score': 0.9977194666862488}]
-----
Yeah, because waking up at 5 AM is exactly what I wanted.
[{'label': 'not_sarcastic', 'score': 0.9990295171737671}]
-----
Fantastic, my phone died right before the interview. Perfect timing.
[{'label': 'not_sarcastic', 'score': 0.9996333122253418}]
-----
I absolutely love waiting in line for an hour.
[{'label': 'not_sarcastic', 'score': 0.9995899796485901}]
-----
Amazing, another update that breaks everything.
[{'label': 'not_sarcastic', 'score': 0.9759174585342407}]
-----
Sure, I’ll totally drop everything just to fix your mistake.
[{'label': 'not_sarcastic', 'score': 0.9990296363830566}]
-----
Wow, what a surprise, the Wi-Fi is terrible again.
[{'label': 'not_sarcastic', 'score': 0.9994483590126038}]
-----
Yeah, because that’s exactly how physics works.
[{'label': 'not_sarcastic', 'score': 0.9990872144699097}]
-----
Perfect, rain on the one day I planned a picnic.
[{

In [ ]:
for text in hard_examples:
    print(text)
    print(sarcasm_clf(text))
    print("-----")

Well, that presentation was… something.
[{'label': 'not_sarcastic', 'score': 0.999167799949646}]
-----
Nice job. Really. Amazing.
[{'label': 'not_sarcastic', 'score': 0.9985752105712891}]
-----
He’s a real genius, isn’t he?
[{'label': 'not_sarcastic', 'score': 0.9994310736656189}]
-----
Guess I should’ve expected that from him.
[{'label': 'not_sarcastic', 'score': 0.9990743398666382}]
-----
Love it when people talk over me.
[{'label': 'not_sarcastic', 'score': 0.9994469285011292}]
-----
What a productive day. I did absolutely nothing.
[{'label': 'not_sarcastic', 'score': 0.9996129870414734}]
-----
Oh awesome, they changed the deadline again.
[{'label': 'not_sarcastic', 'score': 0.997204065322876}]
-----
Yeah, because that’s never happened before.
[{'label': 'not_sarcastic', 'score': 0.997791051864624}]
-----
I’m sure this will end well.
[{'label': 'not_sarcastic', 'score': 0.9993771910667419}]
-----
Great, now I get to redo all my work.
[{'label': 'not_sarcastic', 'score': 0.9924625158